In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn import SAGEConv
from dgl.nn.pytorch import GraphConv, ChebConv, GATConv
import numpy as np
import dask.diagnostics
import fsspec
import xarray as xr
import matplotlib.pyplot as plt
import networkx as nx
import os
from torch.utils.data import DataLoader
import torch.optim as optim
import warnings
import time
import select as sl
import pickle
from load_data import *
from utils import *
from Newmodel import *
import wandb
from fv3net.artifacts.resolve_url import resolve_url
from vcm import get_fs



/home/ebrahimn/miniconda3/envs/dlwp2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
coarsenInd=2
variableList=['h500','h200','h850']
TotalSamples=8500
Chuncksize=500
batch_size=2
window=24
lead=4

data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/" 
state_training_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "atmos_dt_atmos.zarr")), consolidated=True
)
# state_training_data2 = xr.open_zarr(fsspec.get_mapper(os.path.join(data_url, 'sfc_dt_atmos.zarr')))
lat_lon_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "state_after_timestep.zarr"))
)

landSea = xr.open_zarr(
    fsspec.get_mapper(
        "gs://vcm-ml-experiments/default/2022-05-09/baseline-35day-spec-sst/fv3gfs_run/state_after_timestep.zarr"
    )
)
landSea_Mask = landSea.land_sea_mask[1].load()
landSea_Mask = landSea_Mask[:, ::coarsenInd, ::coarsenInd].values.flatten()


lat=(lat_lon_data.latitude[1].load())
lon=(lat_lon_data.longitude[1].load())
lat=lat[:,::coarsenInd,::coarsenInd].values.flatten()
lon=lon[:,::coarsenInd,::coarsenInd].values.flatten()
# cosLat=np.expand_dims(np.cos(lat),axis=1)
# cosLon=np.expand_dims(np.cos(lon),axis=1)
# sinLat=np.expand_dims(np.sin(lat),axis=1)
# sinLon=np.expand_dims(np.sin(lon),axis=1)
cosLat=np.cos(lat)
cosLon=np.cos(lon)
sinLat=np.sin(lat)
sinLon=np.sin(lon)
for i in range(3):
        if i==0:
            sinLon=torch.tensor(sinLon).unsqueeze(0).repeat(int(window/lead),1)
            cosLon=torch.tensor(cosLon).unsqueeze(0).repeat(int(window/lead),1)
            sinLat=torch.tensor(sinLat).unsqueeze(0).repeat(int(window/lead),1)
            cosLat=torch.tensor(cosLat).unsqueeze(0).repeat(int(window/lead),1)
            landSea_Mask=torch.tensor(landSea_Mask).unsqueeze(0).repeat(int(window/lead),1)
        elif i==1:
            sinLon=(sinLon).unsqueeze(0).repeat(1,1,1)
            cosLon=(cosLon).unsqueeze(0).repeat(1,1,1)
            sinLat=(sinLat).unsqueeze(0).repeat(1,1,1)
            cosLat=(cosLat).unsqueeze(0).repeat(1,1,1)
            landSea_Mask=(landSea_Mask).unsqueeze(0).repeat(1,1,1)

        elif i==2:
            sinLon=(sinLon).unsqueeze(0).repeat(batch_size,1,1,1)
            cosLon=(cosLon).unsqueeze(0).repeat(batch_size,1,1,1)
            sinLat=(sinLat).unsqueeze(0).repeat(batch_size,1,1,1)
            cosLat=(cosLat).unsqueeze(0).repeat(batch_size,1,1,1)
            landSea_Mask=(landSea_Mask).unsqueeze(0).repeat(batch_size,1,1,1)

In [ ]:
state_training_data

In [5]:
ss=0
Z500train=state_training_data[variableList[0]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
T2mtrain1=state_training_data[variableList[1]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
T2mtrain2=state_training_data[variableList[2]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()


In [6]:
np.shape(Z500train)

(6, 500, 24, 24)

In [7]:
16*16*6

1536